In [181]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
# nltk.download()

from lib.consts import dataset_dir

train_dataset_path = os.path.join(dataset_dir, '79_rows_text_depth_3.csv')
# Открою датасет
print('Dataset:', train_dataset_path)
data = pd.read_csv(train_dataset_path, sep=",")
# характеристики
print("размер:", data.shape)
print("\nколонки:\n", data.dtypes)


Dataset: datasets\79_rows_text_depth_3.csv
размер: (78, 10)

колонки:
 №                                   int64
ИНН                                 int64
Наименование организации           object
Полное наименование организации    object
Сайт                               object
Индустрия                          object
Даты                               object
Телефоны                           object
Текст                              object
Документы                          object
dtype: object


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dturc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [182]:
data.head()

,№,ИНН,Наименование организации,Полное наименование организации,Сайт,Индустрия,Даты,Телефоны,Текст,Документы
0,7,7705033216,"ОАО ""РОТ ФРОНТ""","ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""РОТ ФРОНТ""",www.rotfront.ru,Пищевая промышленность,,,error ray id aacef utc connection time...,NaN
1,17,5051000880,"АО ""ЩЛЗ""","АКЦИОНЕРНОЕ ОБЩЕСТВО ""ЩЕРБИНСКИЙ ЛИФТОСТРОИТЕЛ...",www.shlz.ru,Машиностроение,,8-800-350-30-50 8-800-350-30-50 8-800-350-30-5...,компания о компании награды и отзывы история ...,NaN
2,26,7724075162,"ФГБУ ""НМИЦ ОНКОЛОГИИ ИМ. Н.Н. БЛОХИНА МИНЗДРАВ...",ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,www.ronc.ru,Медицинская промышленность,30.8.2022 30.8.2022 30.8.2022 30.8.2022 5.11.2...,8-499-324-24-24 8-499-324-25-94 8-499-324-24-2...,фгбу нмиц онкологии им н н блохина минздрава ...,NaN
3,55,7720605108,"ООО ""ФАБРИКА ВЕНТИЛЯЦИИ ГАЛВЕНТ""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ФАБР...",www.ventilyacia.ru,Лёгкая промышленность,6.4.2020 6.1.2020,8-495-790-76-98 8-495-790-76-98 8-495-790-76-9...,к сожалению ваш браузер не поддерживает javas...,NaN
4,80,7724190750,"АО ""СМЕРФИТ КАППА РУС""","АКЦИОНЕРНОЕ ОБЩЕСТВО ""СМЕРФИТ КАППА РУС""",www.smurfitkappa.com/ru,Целлюлозно-бумажная промышленность,3.1.2022 13.1.2022 13.1.2022 13.1.2022 13.1.20...,,перейти к основному содержимому страницыинвес...,NaN


Удалю данные, которые для исследования мне не нужны:


In [183]:
data = data.filter(['Индустрия','Текст'], axis=1)

Удалю латиницу

In [184]:
print("\nПропущенные значения, %:")
for index, value in data.isnull().sum().get(lambda x: x > 0).items():
  print("{:25s} {: ^10} {:>10.3f}%\t{:s}".format(index, value, value*100/data.shape[0], str(data[index].dtype)))


Пропущенные значения, %:
Текст                         7           8.974%	object


In [185]:
data_no_score = data.dropna(subset=["Текст"])
print("Очистка от строк, где Текст = NaN:")
print("Было %d значений. Стало %d. Было удалено %d." % (data.shape[0], data_no_score.shape[0], data.shape[0] - data_no_score.shape[0]))

data = data_no_score

Очистка от строк, где Текст = NaN:
Было 78 значений. Стало 71. Было удалено 7.


Узнаю, сколько текстов в каждой категории

In [186]:
for industry in data['Индустрия'].unique():
  print(industry, '-', str(data[data['Индустрия'] == industry].count()[0]))


Пищевая промышленность - 3
Машиностроение - 4
Медицинская промышленность - 4
Лёгкая промышленность - 12
Целлюлозно-бумажная промышленность - 7
Электротехническая промышленность - 21
Текстильная промышленность - 6
Энергетическая промышленность - 2
Деревообрабатывающая  - 3
Металлообработка - 8
Авиационно-космическая промышленность - 1


In [187]:
data = data.drop(index=0, axis=1)   # единственное дурацкое значение, другие проверил
data['Текст'][:][:6]

1     компания о компании награды и отзывы история ...
2     фгбу нмиц онкологии им н н блохина минздрава ...
3     к сожалению ваш браузер не поддерживает javas...
4     перейти к основному содержимому страницыинвес...
5     продукция каталог сертификаты продажа для кли...
6     уважаемые клиенты отдел торговли завода санте...
Name: Текст, dtype: object

In [188]:
import re
data['Текст'] = data['Текст'].apply(lambda x: re.sub('[a-zA-Z]', '', x))

In [189]:
# сохраню возможность декодирования сюда
cat_names_decoding_map = dict()

for cat in ['Индустрия']:
  le = LabelEncoder()
  lables = le.fit_transform(data[cat])
  data[cat] = le.transform(data[cat])
  cat_names_decoding_map[cat] = {
    'encoder': le,
    'lables': lables,
  }

print(data.dtypes)
data.head()

Индустрия     int32
Текст        object
dtype: object


,Индустрия,Текст
1,3,компания о компании награды и отзывы история ...
2,4,фгбу нмиц онкологии им н н блохина минздрава ...
3,2,к сожалению ваш браузер не поддерживает пожа...
4,8,перейти к основному содержимому страницыинвес...
5,2,продукция каталог сертификаты продажа для кли...


Это тренировочная выборка X

In [190]:
print(data['Текст'][:5])

1     компания о компании награды и отзывы история ...
2     фгбу нмиц онкологии им н н блохина минздрава ...
3     к сожалению ваш браузер не поддерживает  пожа...
4     перейти к основному содержимому страницыинвес...
5     продукция каталог сертификаты продажа для кли...
Name: Текст, dtype: object


Вынес классы в отдельный файл

In [191]:
from lib.StemmedTfidfVectorizer import StemmedTfidfVectorizer, tokenize

TfidfVectorizer со стеммингом

In [192]:
tfidf_vectorizer = StemmedTfidfVectorizer(
  tokenizer=tokenize, 
  analyzer="word", 
  stop_words=stopwords.words("russian"), 
  ngram_range=(1,1), 
  # min_df=3    # limit of minimum number of counts: 3 Установив min_df=2 мы откинем, все слова, которые встречаются менее, чем в 2 документах.
)

In [193]:
# Lookup Error чинится вызовом:
# nltk.download() - в фоне вылезет окошко. Поставить все пакеты
tfidf = tfidf_vectorizer.fit_transform(data['Текст'])
tfidf_vectorizer.vocabulary_

{'компан': 2894,
 'наград': 3887,
 'отзыв': 4771,
 'истор': 2586,
 'сертификат': 6984,
 'разрешен': 6410,
 'лифт': 3345,
 'устройств': 8245,
 'безопасн': 418,
 'двер': 1560,
 'шахт': 8716,
 'мисс': 3678,
 'ценност': 8575,
 'фирмен': 8374,
 'стил': 7489,
 'документ': 1792,
 'подрядчик': 5379,
 'смр': 7160,
 'партнер': 4953,
 'реквизит': 6583,
 'продукц': 6027,
 'пассажирск': 4959,
 'панорамн': 4928,
 'больничн': 567,
 'грузов': 1511,
 'мал': 3453,
 'смарт': 7136,
 'технолог': 7799,
 'инвалидн': 2441,
 'подъемник': 5406,
 'виртуальн': 871,
 'тур': 8000,
 'шоурум': 8775,
 'строительн': 7550,
 'задан': 2062,
 'сер': 6965,
 'ес': 1940,
 'машин': 3520,
 'отделен': 4759,
 'услуг': 8209,
 'сервис': 6967,
 'техподдержк': 7808,
 'монтаж': 3778,
 'продаж': 6010,
 'запасн': 2153,
 'част': 8629,
 'проектирован': 6052,
 'гарантийн': 1312,
 'обслуживан': 4417,
 'пресс': 5838,
 'центр': 8578,
 'сми': 7147,
 'газет': 1289,
 'вверх': 715,
 'фотогалере': 8413,
 'видеогалере': 843,
 'аудиоэкскурс': 339,
 

70 текстов, всего 7860 слов

In [194]:
tfidf.todense().shape

(70, 9030)

In [195]:
clf = MultinomialNB().fit(tfidf, data['Индустрия'])

In [196]:
def get_encoded_industry_name(i):
  return cat_names_decoding_map['Индустрия']['encoder'].inverse_transform([i])[0]

ТЕСТ

In [197]:
docs_new = ['лифты сертификаты на устройства безопасности лифтов', 'изделиячипсы и снекимолочная продукцияэлектроникамодная одеждацветыбакалеясельскохозяйственная продукциязамороженная продукциямебельтовары для здоровья и красотытовары для домашнего хозяйствапромышленные товарымясо рыба и птицаупаковка и бумажная']
X_new_tfidf = tfidf_vectorizer.transform(docs_new)

predicted = map(lambda x: get_encoded_industry_name(x), clf.predict(X_new_tfidf))
for doc, category in zip(docs_new, predicted):
  print('%r => %s' % (doc, category))

'лифты сертификаты на устройства безопасности лифтов' => Электротехническая промышленность
'изделиячипсы и снекимолочная продукцияэлектроникамодная одеждацветыбакалеясельскохозяйственная продукциязамороженная продукциямебельтовары для здоровья и красотытовары для домашнего хозяйствапромышленные товарымясо рыба и птицаупаковка и бумажная' => Электротехническая промышленность


In [207]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

text_clf = Pipeline([
  ('tfidf', StemmedTfidfVectorizer(
    # smooth_idf=True, sublinear_tf=True, use_idf=True,
    max_df=19, min_df=6
  )),
  ('clf', MultinomialNB(
    # alpha=0.1
  )),
])

# mean_test_score = 0.314 | 40s | best={'tfidf__min_df': 5, 'tfidf__smooth_idf': True, 'tfidf__sublinear_tf': True, 'tfidf__use_idf': True}
# parameters = {
#   'tfidf__use_idf': (True, False),
#   'tfidf__smooth_idf': (True, False),
#   'tfidf__sublinear_tf': (True, False),
#   'tfidf__min_df': (3, 5),
# }

# mean_test_score = 0.328 | 14s | best={'tfidf__min_df': 10} 0.32857142857142857
# parameters = {
#   'tfidf__min_df': (5, 6, 7, 8, 9, 10),
# }

# mean_test_score = 0.328 | 25s | {'tfidf__min_df': 10} 0.32857142857142857
# parameters = {
#   'tfidf__min_df': (10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20),
# }

# mean_test_score = 0.37 мало данных для df_max > 10 | 13m | best={'tfidf__max_df': 11, 'tfidf__min_df': 10, 'tfidf__smooth_idf': True, 'tfidf__sublinear_tf': True, 'tfidf__use_idf': True}
# parameters = {
#   'tfidf__min_df': (5, 6, 7, 8, 9, 10),
#   'tfidf__max_df': (5, 7, 8, 9, 10, 11),
#   'tfidf__use_idf': (True, False),
#   'tfidf__smooth_idf': (True, False),
#   'tfidf__sublinear_tf': (True, False),
# }

# mean_test_score = 0.37 мало данных для df_max > 10 | 5m | best={'tfidf__max_df': 6, 'tfidf__min_df': 6} 0.37142857142857144
# parameters = {
#   'tfidf__min_df': (1, 2, 3, 4, 5, 6, 7, 8, 9, 10),
#   'tfidf__max_df': (2, 3, 4, 5, 6, 7, 8, 9, 10, 11),
# }

# mean_test_score = 0.36 | 3m | best={'tfidf__max_df': 20, 'tfidf__min_df': 10} 0.3571428571428571
# parameters = {
#   'tfidf__min_df': (1, 2, 3, 4, 5, 6, 7, 8, 9, 10),
#   'tfidf__max_df': (20, 30, 40, 50, 60, 70, 80),
# }

# mean_test_score = 0.37 | 3.40m | best={'tfidf__max_df': 25, 'tfidf__min_df': 10} 0.37142857142857144
# parameters = {
#   'tfidf__min_df': (10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20),
#   'tfidf__max_df': (25, 30, 40, 50, 60, 70, 80),
# }


# 
# Удалил латиницу
# 


# mean_test_score = 0.39 | 4m | best={'tfidf__max_df': 25, 'tfidf__min_df': 10} 0.38571428571428573
# parameters = {
#   'tfidf__min_df': (10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20),
#   'tfidf__max_df': (25, 30, 40, 50, 60, 70, 80),
# }

# mean_test_score = 0.39 | 1m | best={'tfidf__max_df': 24, 'tfidf__min_df': 10} 0.38571428571428573
# parameters = {
#   'tfidf__min_df': (8, 9, 10, 11, 12),
#   'tfidf__max_df': (20, 21, 22, 23, 24, 25),
# }

# mean_test_score = 0.39 | 1m | best={'tfidf__smooth_idf': True, 'tfidf__sublinear_tf': True, 'tfidf__use_idf': True} 0.38571428571428573
# parameters = {
#   'tfidf__use_idf': (True, False),
#   'tfidf__smooth_idf': (True, False),
#   'tfidf__sublinear_tf': (True, False),
# }


# 
# Собрал больше данных 1,5кБ vs 647
# 


# mean_test_score = 0.33 | 1m | best={'tfidf__max_df': 20, 'tfidf__min_df': 10} 0.3285714285714286
# Это был лучший на том датасете
# parameters = {
#   'tfidf__min_df': (8, 9, 10, 11, 12),
#   'tfidf__max_df': (20, 21, 22, 23, 24, 25),
# }

# mean_test_score = 0.36 | 4m | best={'tfidf__max_df': 15, 'tfidf__min_df': 12} 0.3571428571428571
# parameters = {
#   'tfidf__min_df': (8, 9, 10, 11, 12),
#   'tfidf__max_df': (13, 14, 15, 16, 17, 18, 19),
# }

# mean_test_score = 0.36 | 3m | best={'tfidf__max_df': 15, 'tfidf__min_df': 13} 0.35714285714285715
# parameters = {
#   'tfidf__min_df': (11, 12, 13),
#   'tfidf__max_df': (13, 14, 15, 16, 17, 18, 19),
# }

# mean_test_score = 0.37 | 30s | best={'clf__alpha': 0.1} 0.37142857142857144
# parameters = {
#   'clf__alpha': (1, 1e-1, 1e-2, 1e-3),
# }

# mean_test_score = 0.37 | 2m | best={'clf__alpha': 0.1} 0.37142857142857144
# parameters = {
#   'clf__alpha': (0.6, 0.5, 0.4, 0.3, 0.2, 1e-1, 0.09, 0.08, 0.07, 0.06),
# }


# 
# Собрал больше данных 1,5мБ vs 1,5кБ
# 


# mean_test_score = 0.47 | 4m | best={'tfidf__max_df': 19, 'tfidf__min_df': 8} 0.4714285714285714
# parameters = {
#   'tfidf__min_df': (8, 9, 10, 11, 12),
#   'tfidf__max_df': (13, 14, 15, 16, 17, 18, 19),
# }


# mean_test_score = 0.5 | 4m | best={'tfidf__max_df': 19, 'tfidf__min_df': 6} 0.5
# parameters = {
#   'tfidf__min_df': (2, 3, 4, 5, 6, 7, 8),
#   'tfidf__max_df': (19, 20, 21, 22, 23),
# }

# mean_test_score =  | | best=
parameters = {
  'clf__alpha': (0.6, 0.5, 0.4, 0.3, 0.2, 1e-1, 0.09, 0.08, 0.07, 0.06),
  'tfidf__use_idf': (True, False),
  'tfidf__smooth_idf': (True, False),
  'tfidf__sublinear_tf': (True, False),
}


# -1 - все ядра ЦП
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1) 

In [208]:

%time
gs_clf = gs_clf.fit(data['Текст'], data['Индустрия'])

CPU times: total: 0 ns
Wall time: 0 ns


c:\Users\dturc\source\repos\Denactive\mch2022-backend\Python\venv\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [206]:
print(gs_clf.best_params_, gs_clf.best_score_)
print(gs_clf.cv_results_['mean_test_score'])


{'tfidf__max_df': 19, 'tfidf__min_df': 6} 0.5
[0.5        0.45714286 0.47142857 0.44285714 0.5        0.47142857
 0.47142857 0.5        0.45714286 0.45714286 0.45714286 0.5
 0.48571429 0.47142857 0.48571429 0.45714286 0.47142857 0.47142857
 0.5        0.5        0.45714286 0.48571429 0.45714286 0.47142857
 0.47142857 0.5        0.47142857 0.45714286 0.48571429 0.45714286
 0.45714286 0.45714286 0.5        0.48571429 0.48571429]


ТЕСТ

In [209]:
docs_new = ['лифты сертификаты на устройства безопасности лифтов', 'изделиячипсы и снекимолочная продукцияэлектроникамодная одеждацветыбакалеясельскохозяйственная продукциязамороженная продукциямебельтовары для здоровья и красотытовары для домашнего хозяйствапромышленные товарымясо рыба и птицаупаковка и бумажная']
X_new_tfidf = tfidf_vectorizer.transform(docs_new)

predicted = map(lambda x: get_encoded_industry_name(x), gs_clf.predict(docs_new))
for doc, category in zip(docs_new, predicted):
  print('%r => %s' % (doc, category))

'лифты сертификаты на устройства безопасности лифтов' => Электротехническая промышленность
'изделиячипсы и снекимолочная продукцияэлектроникамодная одеждацветыбакалеясельскохозяйственная продукциязамороженная продукциямебельтовары для здоровья и красотытовары для домашнего хозяйствапромышленные товарымясо рыба и птицаупаковка и бумажная' => Электротехническая промышленность


In [211]:
import joblib
import os
from lib.consts import cache_dir

joblib.dump((cat_names_decoding_map['Индустрия']['encoder'], gs_clf), os.path.join(cache_dir, 'predict_industry_model.dat'))

['cache\\predict_industry_model.dat']